# Proximity Measures: Similarity, Dissimilarity, Distance

* __Author: Prof. Nagiza F. Samatova__
* __Email: samatova@csc.ncsu.edu__
* __Date: October 3, 2018__

In [1]:
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity 
# http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise

from scipy.stats import pearsonr as _pearson_cor
# https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.pearsonr.html

from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html

In this lesson, we will cover ___proximity measures___, including:
+ _similarity_ measures
+ _dissimilarity_ measures, and 
+ _distance_ metrics. 

Measuring the proximity between two objects is an important step in many machine learning and data mining tasks including but not limited to:
+ clustering
+ nearest neighbor classification
+ missing links prediction
+ community detection in social networks
+ recommendation systems such as collaborative filtering

> The proximity between two objects is a numerical measure of the degree to which the two objects are alike. 

## Similarity vs. Dissimilarity vs. Distance

Proximity often refers to a similarity or dissimilarity:
+ __Similarity__:
    - Numerical measure of how alike two data objects are.
    - Is higher when objects are more alike.
    - Often falls in the range [0,1]:
    - Examples: Cosine, Jaccard, Tanimoto,
+ __Dissimilarity__:
    - Numerical measure of how different two data objects are
    - Lower when objects are more alike
    - Minimum dissimilarity is often 0
    - Upper limit varies
+ __Distance__:
    - A special class of dissimilarities with some particular mathematical properties
    - Ranges from 0 to positive infinity
    - Value of 0 means no differences between two objects

+ The _similarity_ between two objects is a numerical measure of the degree to which the two objects are alike. Similarities are higher for pairs of objects that are more alike, and often range from 0 (no similarity) to 1 (complete similarity).
+ The _dissimilarity_ between two objects is a numerical measure of the degree to which the two objects are different. Dissimilarities are lower for more similar pairs of objects, and sometimes range from 0 (no dissimilarity) to 1 (complete dissimilarity)
+ Similarities can often be transformed to dissimilarities in a straightforward way:
    - If the similarity ($s$) falls in the interval $[0,1]$, then the dissimilarity ($d$) can be defined as:
    > $d = 1 - s$
+ _Distance_ is a special class of dissimilarities with some particular mathematical properties. Distances usually range from 0 to infinity.    

### Similarity: Properties

If $s(x,y)$ is the _similarity_ between points $x$ and $y$, 
then the typical properties of similarities are the following: 
+ $s(x,y) = 1$ only if $x = y$ 
+ $0 <= s <= 1$, and 
+ $s(x,y) = s(y,x)$ for all $x$ and $y$ (_Symmetry_).

## Similarity for Binary (0, 1) Data

Similarity measures for objects with only _binary_ attributes (e.g., binary vectors): 
+ _SMC_: Simple Matching Coefficient
+ _Jaccard_ coefficient:
    - Ignores zero matches
    - Desirable when two records should not be similar because a large number of characteristics are absent in both:
        * Document-document similarity: matching Words used
        * User-User similarity: matching Items purchased
+ _Tanimoto_ coefficient

In [ ]:
First, define two binary vectors: 

In [2]:
v1 = np.array((1,0,0,0,1,0,1,0,0,0))
v2 = np.array ((0,0,1,0,0,1,0,0,0,0))
print (v1)
print (v2)
sum (v1 == v2)

[1 0 0 0 1 0 1 0 0 0]
[0 0 1 0 0 1 0 0 0 0]


5

The ___simple matching coefficient___ (__SMC__) for two binary vectors is given by the number of attributes (i.e., vector components) where both vectors have the same value (i.e., both have a value of 0 or both have a value of 1) over the total number of attributes.

In [3]:
def smc (b1, b2):
    return sum( b1 == b2) / b1.size

In [4]:
smc (v1, v2)

0.5

The ___Jaccard___ coefficient for two binary vectors is given by the number of attributes (i.e., vector components) where both vectors have a value of 1 over the number of attributes where at least one of the vectors has a value of 1.

In [5]:
def jaccard (b1, b2):
    return sum((b1==1)&(b2==1))/sum((b1==1)|(b2==1))

In [16]:
jaccard (v1, v2)

0.0

As you can see, the similarities obtained for $v1$ and $v2$ using the SMC and the Jaccard coefficient are very different:
+ This is because the SMC measures similarity in terms of __both__ matching 0's and matching 1's, while the Jaccard coefficient only considers matching 1s. 
+ As a result, the Jaccard coefficient is frequently used to handle objects consisting of _asymmetric_ binary distributions:
    - for example, customer transactions at a store, where the matching 0's (i.e., store items not bought in neither of the transactions) are likely to greatly outnumber the matching 1's (i.e., store items bought in both transactions).

Because the Jaccard coefficient ranges between 0 and 1, then the dissimilarity between $v1$ and $v2$ based on their Jaccard coefficient can be defined as:
> $1 - $ jaccard $(v1, v2)$

In [6]:
1- jaccard (v1, v2)

1.0

## Similarity for Non-Binary Data

Similarity measures for objects with _non-binary_ (i.e., discrete or continuous) attributes: 
+ _Cosine_ similarity
+ _Pearson_ correlation
+ _Spearman_ correlation

### Cosine Similarity

___Cosine___ similarity is a measure of similarity between two vectors given by the cosine of the angle between them:
+ The cosine similarity between two vectors can be computed by dividing their scalar product by the product of their norms:
> $cos (x, y) = \dfrac{(x, y)} { ||x|| \times ||y||}$
+ Note that cosine similarity _compares two vectors only in terms of their directions_, and not their norms (i.e., magnitudes). 
+ Also note that cosine similarity ranges from -1 to 1:
    + __1__ if the two vectors have the same direction, 
    + __-1__ if they have opposite directions, and 
    + __0__ if they are orthogonal (i.e., perpendicular) to each other.

Cosine similarity is commonly used in information retrieval and text mining to compute the similarity between two documents:
+ Documents are often represented as vectors, where each attribute represents the frequency with which a particular term (word) occurs in the document.
+ These documents are generally very sparse (i.e., have relatively few non-zero attributes out of thousands or tens of thousands of attributes). 
+ Therefore, as with customer transactions, similarity between documents must not consider matching 0's; otherwise, most documents would be highly similar. 
+ The use of cosine similarity is appropriate in this case because:
    - like the Jaccard coefficient, it does not consider matching 0's when computing the similarity between two objects, but, 
    - unlike the Jaccard coefficient, it can also handle non-binary attributes.

In [7]:
import numpy as np

def cos_similarity (x, y):
    """Takes 2 vectors x, y and returns the cosine similarity according 
    to the definition of the dot product
    """
    dot_product = np.inner(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    return dot_product / (norm_x * norm_y)    

In [8]:
v3 = [2,1,0,0,5,0,2,0,0,0]
v4 = [1,0,3,0,2,1,3,0,0,0]
print (v3)
print (v4)

[2, 1, 0, 0, 5, 0, 2, 0, 0, 0]
[1, 0, 3, 0, 2, 1, 3, 0, 0, 0]


In [9]:
cos_similarity(v3,v4)

0.6301260378126045

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

# note that 1D arrays is not passable
cosine_similarity([v3], [v4])

array([[0.63012604]])

Note that passing one dimension arrays as input data is deprecated in sklearn version 0.17, and will raise ValueError:

In [11]:
# Should generate ValueError
try:
    cosine_similarity(v3, v4)
except ValueError as e:
    print ('Unable to pass one-dimensional array : {} \n'.format(e))

Unable to pass one-dimensional array : Expected 2D array, got 1D array instead:
array=[2. 1. 0. 0. 5. 0. 2. 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample. 



In [12]:
# Another option:

from scipy import spatial

result = 1 - spatial.distance.cosine(v3, v4)
result

0.6301260378126045

#### Performance Issues

* https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists

### Pearson and Spearman Correlation

___Pearson___ correlation measures the _linear_ relationship between objects:

+ Strictly speaking, Pearson’s correlation requires that each dataset be normally distributed. 
+ It ranges between -1 and +1 with:
    - 0 implying no correlation. 
    - Correlations of -1 or +1 imply an exact linear relationship. 
+ Positive correlations imply that as x increases, so does y. 
+ Negative correlations imply that as x increases, y decreases.
+ The $p$-value of the correlation roughly indicates the probability of an uncorrelated system producing datasets that have a Pearson correlation at least as extreme as the one computed from these datasets. 
    - The $p$-values are not entirely reliable but are probably reasonable for datasets larger than 500 or so.

To compute correlation, we standardize data objects, $p$ and $q$ to their $Z$-scores by subtracting the mean and dividing by standard diviation, and then take the dot product of their $Z$-scores:
```python
pZ = (p - mean(p))/sd(p)
qZ = (q - mean(q))/sd(q)
cor(p, q) = (pZ, qZ)
```

In [13]:
a = [15, 12, 8, 8, 7, 7, 7, 6, 5, 3]
b = [10, 25, 17, 11, 13, 17, 20, 13, 9, 15]

In [14]:
# Option-0:

def corr(u, v):
    "u & v should be numpy arrays."
    u = np.array(u)
    v = np.array(v)
    mean1 = u.mean() 
    mean2 = v.mean()
    std1 = u.std()
    std2 = v.std()

#     corr = ((data1-mean1)*(data2-mean2)).mean()/(std1*std2)
    corr = ((u*v).mean()-mean1*mean2)/(std1*std2)
    return corr

In [67]:
corr (a, b)

0.14499815458068538

In [15]:
# Option-1
import math

def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)

def pearson_cor(x, y):
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff

    return diffprod / math.sqrt(xdiff2 * ydiff2)

In [16]:
pearson_cor (a,b)

0.14499815458068516

In [17]:
# Option-2:
from scipy.stats import pearsonr as _pearson_cor

cor, pval = _pearson_cor(a, b)
print (cor)
print (pval)

0.14499815458068518
0.6894014481166955


In [18]:
# Option-3:
from scipy.stats import linregress

linregress(a, b)

LinregressResult(slope=0.20833333333333331, intercept=13.375, rvalue=0.14499815458068518, pvalue=0.689401448116695, stderr=0.5026170462708364)

In [42]:
import numpy
numpy.corrcoef?

In [45]:
# Option-4:
import numpy

numpy.corrcoef(a, b)[0, 1]

0.14499815458068518

In [19]:
# Option-5:
import pandas as pd

a = [15, 12, 8, 8, 7, 7, 7, 6, 5, 3]
b = [10, 25, 17, 11, 13, 17, 20, 13, 9, 15]

data = list (zip(a, b))
print (data)

df = pd.DataFrame(data)
df.corr()

[(15, 10), (12, 25), (8, 17), (8, 11), (7, 13), (7, 17), (7, 20), (6, 13), (5, 9), (3, 15)]


,0,1
0,1.000000,0.144998
1,0.144998,1.000000


## Distance

___Distance___ is a special class of dissimilarities with some particular mathematical properties. Distances usually range from 0 (identity between two objects) to positive infinity.

> If $d(x,y)$ is the distance between points $x$ and $y$, then the following properties hold: 
+ $d(x,y) >= 0$ for all $x$ and $y$, 
+ $d(x,y) = 0$ only if $x = y$ (Positivity),
+ $d(x,y) = d(y,x)$ for all $x$ and $y$ (Symmetry), and 
+ $d(x,z) <= d(x,y) + d(y,z)$ for all $x$, $y$, and $z$ (Triangle Inequality).

+ Distance measures for objects with continuous attributes include:
    - _Euclidean_ distance:
        * Attributes must be standarded to $Z$-scores if their scales differ!
    - _Minkowski_ distance:
        * Manhattan (city block) distance
    - _Mahalanobis_ distance:
        * Makes attributes that are highly correlated with other attributes not to contribute as much to the distance
+ Distance measures for objects with categorical attributes include:
    - _Hamming_ distance:
        * The distance is 0 if the attributes are in the same category and 1, otherwise
        * Measures the number of bits that are different between two binary vectors

### Euclidean Distance

The most commonly used distance metric is the Euclidean distance. The Euclidean distance between two vectors (in one-, two-, three-, or any higher “n”-dimensional space) is given by the square root of the sum of the squared differences between the corresponding vector components:
> $dist (x, y) = \sqrt {\sum (x_k - y_k)^2}$

+ https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy
+ http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise

In [23]:
# Option-1

def eucl_dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
dist = eucl_dist(a,b)

dist

5.196152422706632

In [25]:
# Option-2
from  numpy.linalg import norm

a = (1, 2, 3)
b = (4, 5, 6)
dist = np.linalg.norm(np.array (a) - np.array(b))
dist

5.196152422706632

In [26]:
# Option-3:
from scipy.spatial import distance
a = (1, 2, 3)
b = (4, 5, 6)
dist = distance.euclidean(a, b)
dist

5.196152422706632

## Summary

Finally, always keep in mind that the proximity measure you use “should fit the type of data. For many types of dense, continuous data, metric distance measures such as Euclidean distance are often used. For sparse data, which often consists of asymmetric attributes, we typically employ similarity measures that ignore 0-0 matches, such as the Jaccard coefficient or cosine similarity. Conceptually, this reflects the fact that, for a pair of complex objects, similarity depends on the number of characteristics they both share, rather than the number of characteristics they both lack.